In [1]:
import glob
import pandas as pd
import numpy as np
import os
import scipy
import random
import librosa
import librosa.display
import random
import matplotlib.pyplot as plt
from keras.models import load_model 

Using TensorFlow backend.


In [ ]:
path_to_validation = '../data_cut/'

def audios_to_spectograms(file_names):
  # Save different shapes in a set
  x, shapes = [], set()

  # Enumerate for logging
  for indx, audio_file in enumerate(file_names):
    # Use mfcc algorithm for spectograms
    signal, sampling_rate = librosa.load(audio_file) 
    matrix = librosa.feature.mfcc(signal, sampling_rate)

    x.append(matrix)
    shapes.add(matrix.shape)
    if (indx+1) % 50 == 0: print('{} analyzed out of {}'.format(indx+1, len(file_names))) # Log progress
    
  return x, shapes


def choose_max_shapes(shapes):
  # Iterate over shapes and choose biggest possible width and height
  w, h = 0, 0
  for shape in shapes:
    w = max(w, shape[0])
    h = max(h, shape[1])
  return w, h

def pad_spectogram(matrix):
  # Since width is always 20 in mfcc, we only check for height difference
  if matrix.shape[1] < def_h:
    diff = def_h - matrix.shape[1]
    # Append half of the difference in beginning
    matrix = np.append(np.zeros((matrix.shape[0], diff//2), dtype=float), matrix, axis=1)
    #Append res in the end
    matrix = np.append(matrix, np.zeros((matrix.shape[0], diff - diff//2), dtype=float), axis=1)
  return matrix



def list_of_audios(dir_path):
  arr = glob.glob(dir_path + '*.wav')
  random.shuffle(arr) # Shuffled data is better for training
  return list(map(lambda x: (x, parse_number(x)), arr))


def parse_number(file_path):
  return int(''.join(ch for ch in list(file_path) if ch.isdigit()))



model = load_model('model.h5')

t_df = pd.DataFrame(list_of_audios(path_to_validation), columns = ['file_name', 'correct'])
print(t_df)
t_matrices, t_shapes = audios_to_spectograms(t_df['file_name'])
print('Different shapes:', t_shapes)
def_w, def_h = choose_max_shapes(t_shapes)
print('Every spectogram should be size of:', (def_w, def_h))
t_x = np.array([pad_spectogram(matrix) for matrix in t_matrices])
print(t_x.shape, '\n')
t_x_r = t_x.reshape(*t_x.shape, 1)

final, prediction = [], model.predict(t_x_r)
for i, pred in enumerate(prediction):
  final.append([t_df['file_name'][i]] + [np.argmax(pred) + 1] + list(map(lambda s: str(s / sum(pred)), pred)))

final_df = pd.DataFrame(final, columns = ['file_name', 'predicted', '1', '2', '3', '4', '5'])
print(final_df)
final_df.to_csv('results.csv')